In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split

In [10]:
# Define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = self.data.iloc[idx, :-1].values.astype(float)
        label = self.data.iloc[idx, -1].astype(float)
        return {'features': torch.tensor(features), 'label': torch.tensor(label)}

csv_file = 'HandMCout.csv'  # Replace 'data.csv' with the path to your CSV file
dataset = CustomDataset(csv_file)

print(dataset[0])

# Split dataset into train and test sets
train_set, test_set = train_test_split(dataset, test_size=0.2, shuffle=True)
print(train_set[0:5])



{'features': tensor([5., 4., 2., 4., 3., 3., 4., 4., 4., 2.], dtype=torch.float64), 'label': tensor(0.0060, dtype=torch.float64)}
[{'features': tensor([7., 2., 5., 2., 3., 1., 8., 3., 9., 1.], dtype=torch.float64), 'label': tensor(0.1050, dtype=torch.float64)}, {'features': tensor([ 3.,  4., 12.,  1., 14.,  1., 12.,  3., 14.,  4.], dtype=torch.float64), 'label': tensor(0.8560, dtype=torch.float64)}, {'features': tensor([ 6.,  4., 14.,  1.,  5.,  3.,  7.,  1., 10.,  3.], dtype=torch.float64), 'label': tensor(0.5430, dtype=torch.float64)}, {'features': tensor([10.,  1.,  4.,  2.,  3.,  2.,  8.,  2., 11.,  3.], dtype=torch.float64), 'label': tensor(0.2270, dtype=torch.float64)}, {'features': tensor([ 2.,  3.,  6.,  1., 12.,  1., 10.,  4.,  3.,  4.], dtype=torch.float64), 'label': tensor(0.0370, dtype=torch.float64)}]


In [ ]:
# Define the neural network
class FunctionApproximator(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FunctionApproximator, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Load CSV fil
# Define model, loss function, and optimizer
input_dim = len(train_set[0]['features'])
output_dim = 1  # Assuming a single output
hidden_dim = 32 # Adjust according to your problem
model = FunctionApproximator(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
batch_size = 128
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for batch in train_loader:
        features, labels = batch['features'], batch['label']
        optimizer.zero_grad()
        outputs = model(features.float())
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
test_loss = 0
with torch.no_grad():
    for batch in test_loader:
        features, labels = batch['features'], batch['label']
        outputs = model(features.float())
        test_loss += criterion(outputs.squeeze(), labels.float()).item()

test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}')

Epoch [1/50], Loss: 0.0582
Epoch [2/50], Loss: 0.0587
Epoch [3/50], Loss: 0.0561
Epoch [4/50], Loss: 0.0671
Epoch [5/50], Loss: 0.0539
Epoch [6/50], Loss: 0.0583
Epoch [7/50], Loss: 0.0669
Epoch [8/50], Loss: 0.0434
Epoch [9/50], Loss: 0.0535
Epoch [10/50], Loss: 0.0419
Epoch [11/50], Loss: 0.0470
Epoch [12/50], Loss: 0.0430
Epoch [13/50], Loss: 0.0320
Epoch [14/50], Loss: 0.0349
Epoch [15/50], Loss: 0.0339
Epoch [16/50], Loss: 0.0265
Epoch [17/50], Loss: 0.0275
Epoch [18/50], Loss: 0.0317
Epoch [19/50], Loss: 0.0304
Epoch [20/50], Loss: 0.0268
Epoch [21/50], Loss: 0.0346
Epoch [22/50], Loss: 0.0258
Epoch [23/50], Loss: 0.0241
Epoch [24/50], Loss: 0.0279
Epoch [25/50], Loss: 0.0259
Epoch [26/50], Loss: 0.0203
Epoch [27/50], Loss: 0.0217
Epoch [28/50], Loss: 0.0244
Epoch [29/50], Loss: 0.0274
Epoch [30/50], Loss: 0.0194
Epoch [31/50], Loss: 0.0245
Epoch [32/50], Loss: 0.0228
Epoch [33/50], Loss: 0.0193
Epoch [34/50], Loss: 0.0268
Epoch [35/50], Loss: 0.0177
Epoch [36/50], Loss: 0.0216
E

In [17]:
manualinput = torch.tensor([[2,3,6,2,11,4,11,1,4,3]]).float()   # answer is 0.044
greathand = torch.tensor([[14,1,14,2,13,4,12,2,11,5]]).float()

model.eval()
with torch.no_grad():
    output = model(manualinput)
print(output)
with torch.no_grad():
    output = model(greathand)
print(output)

tensor([[0.0418]])
tensor([[1.0145]])
